In [1]:
query = "Qatar"
users = ""
topic = "HIA"
subtopic = ""
query_language = "All"
target_languages = "English,Arabic"
num_tweets = 50
days = 7
result_type = "recent"

In [2]:
LANGUAGE_CODES={"All":"","Afrikaans":"af","Arabic":"ar","Assamese":"as","Bangla":"bn","Bosnian(Latin)":"bs","Bulgarian":"bg","Cantonese(Traditional)":"yue","Catalan":"ca","Chinese Simplified":"zh-Hans","Chinese Traditional":"zh-Hant","Croatian":"hr","Czech":"cs","Dari":"prs","Danish":"da","Dutch":"nl","English":"en","Estonian":"et","Fijian":"fj","Filipino":"fil","Finnish":"fi","French":"fr","German":"de","Greek":"el","Gujarati":"gu","Haitian Creole":"ht","Hebrew":"he","Hindi":"hi","Hmong Daw":"mww","Hungarian":"hu","Icelandic":"is","Indonesian":"id","Irish":"ga","Italian":"it","Japanese":"ja","Kannada":"kn","Kazakh":"kk","Klingon":"tlh-Latn","Klingon(plqaD)":"tlh-Piqd","Korean":"ko","Kurdish(Central)":"ku","Kurdish(Northern)":"kmr","Latvian":"lv","Lithuanian":"lt","Malagasy":"mg","Malay":"ms","Malayalam":"ml","Maltese":"mt","Maori":"mi","Marathi":"mr","Norwegian":"nb","Odia":"or","Pashto":"ps","Persian":"fa","Polish":"pl","Portuguese(Brazil)":"pt-br","Portuguese(Portugal)":"pt-pt","Punjabi":"pa","Queretaro Otomi":"otq","Romanian":"ro","Russian":"ru","Samoan":"sm","Serbian(Cyrillic)":"sr-Cyrl","Serbian(Latin)":"sr-Latn","Slovak":"sk","Slovenian":"sl","Spanish":"es","Swahili":"sw","Swedish":"sv","Tahitian":"ty","Tamil":"ta","Telugu":"te","Thai":"th","Tongan":"to","Turkish":"tr","Ukrainian":"uk","Urdu":"ur","Vietnamese":"vi","Welsh":"cy","Yucatec Maya":"yua"}
topic = topic.lower()
# username = user
userslist = users.split(',')
query_language = LANGUAGE_CODES.get(query_language, "")
target_languages = [LANGUAGE_CODES.get(lang, "") for lang in target_languages.split(",")]
if "en" not in target_languages:
  target_languages.append("en") # we always include english in target languages
num_tweets = int(num_tweets)
max_days = int(days)

In [3]:
assert (query == "") or (users == "") # Can either search by query or by user, not both

In [4]:
# Cosmos Temp
COSMOS_URL = "https://cosmos-socialmedia-dev-01.documents.azure.com:443/"
COSMOS_KEY = "k4nj8hc0R0hVBd888i1CoB3Pkv3aimDG1eE8BRNat54POhxiQaggqP251yHaPE9bzQwVXqtSUELZKfIaRfvAQg=="
COSMOS_DATABASE_NAME = "databasesocialmedia01"
COSMOS_CONTAINER_NAME = "KSA_Tweets"


# Text Analytics Temp
TEXT_ANALYTICS_KEY = "dd902b6fa8bb429aac0bba75532d456e"
TEXT_ANALYTICS_ENDPOINT = "https://textanalytics-socialmedia-dev-01.cognitiveservices.azure.com/"
TEXT_ANALYTICS_REGION = "westeurope"

#Translator
TRANSLATOR_KEY = "7d5eef61105d44a78840f7a8e49a8d9b"
TRANSLATOR_ENDPOINT = "https://api.cognitive.microsofttranslator.com/"
TRANSLATOR_REGION = "westeurope" 

# Twitter Temp
TWITTER_API_KEY = "3pJCEcL9129CXWzFNg0mRSIFC"
TWITTER_API_SECRET_KEY = "c8z6vrQLqu8pfnfaWIK4qksgfqS7hrvtDlstbIP1l4iAqx7WFQ"
TWITTER_ACCESS_TOKEN = "1621038712728571905-suTbQDs30WjsiArwqQDcMXf1lITkHM"
TWITTER_ACCESS_TOKEN_SECRET = "nUibZQYC7prXLImzoPDFENaoBB1xxA0i1UqBMv3TawytW"
TWITTER_ACCESS_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAJjToAEAAAAA1XtTjINXfjBjqammKhVfECdJqus%3D1llMZPdGaggEZect52d5KtjdjyyRBXdN39OHNVMCjox2yL98Id"

In [5]:
# Azure Maps Temp
MAPS_KEY = "-UbQBKw-d24t4ZfppLDxHgTYVbXm63ZKg24V2xt9NHY"

In [6]:
import re
import sys
import copy
import tweepy
import urllib.parse
import sys, time, json, requests, uuid

from tweepy import API
from tweepy import Cursor
from tweepy import OAuthHandler

from dateutil.parser import parse
from datetime import datetime, date, timedelta # don't import time here. It messes with the default library


from azure.cosmos import CosmosClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

In [7]:
def authenticate_client():
    """
    Returns: text analytics client
    """
    ta_credential = AzureKeyCredential(TEXT_ANALYTICS_KEY)
    text_analytics_client = TextAnalyticsClient(
            endpoint=TEXT_ANALYTICS_ENDPOINT, 
            credential=ta_credential)
    return text_analytics_client
text_analytics_client = authenticate_client()
# TODO: Add opinion mining
def get_sentiment(inp_text):
    #Parameters: 
    #  inp_text: text to analyze
    #Returns:
    #  sentiment, sentiment score
    documents = [inp_text]
    response = text_analytics_client.analyze_sentiment(documents = documents)[0]  
    try:
        overallscore = response.confidence_scores.positive + (0.5*response.confidence_scores.neutral) # check logic of this
        return response.sentiment, overallscore
    except Exception as err:
        print("Encountered Sentiment exception. {}".format(err))
        return "Neutral",0.5
def get_ner(inp_text):
    #Parameters: 
    #  inp_text: text to analyze
    #Returns:
    #  NER Results as a list of dictionaries with keys: text, category, subcategory, length, offset, confidence
    try:
        documents = [inp_text]
        result = text_analytics_client.recognize_entities(documents = documents)[0]  
        return [{"text": x.text, "category": x.category, "subcategory": x.subcategory, "length": x.length, "offset": x.offset, "confidence_score": x.confidence_score} for x in result.entities]
    except Exception as err:
        print("Encountered NER exception. {}".format(err))
    return []

def get_routes_from_text(inp_text):
    try:
        documents = [inp_text]
        result = text_analytics_client.recognize_entities(documents=documents)[0]

        # Filter entities that are of type 'Location'
        location_entities = [x for x in result.entities if x.category == 'Location' and x.subcategory == 'GPE']

        # Initialize route variables
        from_entity = None
        to_entity = None
        routes = []

        # Find consecutive 'from' and 'to' entities to create route tuples
        for entity in location_entities:
            entity_position = entity.offset
            entity_length = entity.length
            entity_text = inp_text[entity_position: entity_position + entity_length]

            if from_entity is None:
                from_entity = entity_text
            elif to_entity is None and entity_position > from_entity_offset:
                to_entity = entity_text
                route = (from_entity, to_entity)
                routes.append(route)
                from_entity = entity_text  # Update from_entity for multi-word locations
                to_entity = None

            from_entity_offset = entity_position

        return routes

    except Exception as err:
        print("Encountered NER exception. {}".format(err))

    return []


def get_key_phrases(inp_text):
    #Parameters: 
    #  inp_text: text to analyze
    #Returns:
    #  List of key phrases
    try:
      documents = [inp_text]
      response = text_analytics_client.extract_key_phrases(documents = documents)[0] 
      if not response.is_error:
          return response.key_phrases
      else:
          print(response.id, response.error)
    except Exception as err:
      print("Encountered Translation exception. {}".format(err))
    return []

In [8]:
def update_cosmos(objects, container): # insert tweets/users to cosmos
    for obj in objects:
        if obj:
            response = container.upsert_item(body=obj) # use upsert so that insert or update
    print("Inserted data to cosmos")

In [9]:
# Tweet Entities processing
import requests
import aiohttp
import asyncio
import json

def get_maps_response(inp):
    url = "https://atlas.microsoft.com/search/fuzzy/json?&subscription-key="+MAPS_KEY+"&api-version=1.0&language=en-US&query="+inp
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()

In [10]:
def get_translation(inp_text, to_languages):
    """
    Params:
    inp_text: text to be translated
    to_languages: list of languages to translate to
    Returns: {lang_code: translation}, language code of the original text
    Call to translator cognitive service detects language and translates to the target languages. 
    Result is a dictionary of language codes to translated text, along with the language detected.
    """
    # Translator setup
    translator_path = "/translate"
    translator_url = TRANSLATOR_ENDPOINT + translator_path
    params = {
    "api-version": "3.0",
    "to": to_languages
    }
    headers = {
    'Ocp-Apim-Subscription-Key': TRANSLATOR_KEY,
    'Ocp-Apim-Subscription-Region': TRANSLATOR_REGION,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
    }
    # create and send request
    body = [{
    'text': inp_text
    }]
    request = requests.post(translator_url, params=params, headers=headers, json=body)
    response = request.json()
    # only ever one string sent for translation, so only list of length 1
    try:
        from_language = response[0]["detectedLanguage"]["language"]
        translations = response[0]["translations"]
        res = {} # dict with language as key and translated text as value e.g. {"ar": "arabic text"}
        for trans in translations:
            res[trans['to']] = trans['text']
        #res[from_language] = inp_text # also include the original text and language in translations - overkill?
        return res, from_language # return the translated text, as well as the language it was translated from
    except Exception as err:
        print("Encountered an exception. {}".format(err))
        return None, None

In [11]:
client = CosmosClient(COSMOS_URL, {'masterKey': COSMOS_KEY})
database = client.get_database_client(COSMOS_DATABASE_NAME)

tweet_container_client = database.get_container_client(container=COSMOS_CONTAINER_NAME)

In [12]:
consumer_key = TWITTER_API_KEY
consumer_secret = TWITTER_API_SECRET_KEY
access_token = TWITTER_ACCESS_TOKEN
access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
bearer_token = TWITTER_ACCESS_BEARER_TOKEN

In [13]:
tweepy_client =tweepy.Client(bearer_token=bearer_token, consumer_key=consumer_key, consumer_secret=consumer_secret, access_token=access_token, access_token_secret=access_token_secret, wait_on_rate_limit=True)


In [14]:
# Set the since_datetime to be 7 days in the past from the current date and time
since_datetime = datetime.utcnow() - timedelta(days=7)

# Get the top 1 tweet from Cosmos DB that meets the criteria
cosmos_query = f"SELECT TOP 1 c.originalid, c.created_at FROM c WHERE c.document_type = 'tweet' and c.query= '{query}' and c.created_at > '{since_datetime.isoformat()}' ORDER BY c.created_at DESC"
items = list(tweet_container_client.query_items(cosmos_query, enable_cross_partition_query=True)) 

In [15]:
# Retrieve the tweet ID if available
if items:
    since_id = str(items[0]['originalid'])
else:
    # Handle the case when no valid tweet ID is available
    since_id = None

In [16]:
regions = ["Africa","Arabian Gulf","Asia","Central America","Europe","Middle East","North America","Oceania","South America"]

In [17]:
def remove_emojis(data):
    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", re.UNICODE)
    return re.sub(emoji, '', data)

In [18]:
query_language

''

In [19]:
def build_entry(topic, status, query_search, container, query_language=None, target_languages=[], username=""):
    tweet = status
    id_str = str(int(tweet["id"]) + abs(hash(topic)))
    # Check for tweets which have already been added
    search_query = 'select * from items where items.id="{0}"'.format(id_str)
    items = list(container.query_items(search_query, enable_cross_partition_query=True))
    dt2 = datetime.now()
    ts = int(time.mktime(dt2.timetuple()))
    at = dt2.strftime("%m/%d/%Y, %H:%M:%S %Z")
    tweet['inserted_to_CosmosDB_at'] = at
    tweet['inserted_to_CosmosDB_ts'] = ts
    new_tweet = True
    if len(items) > 0:
        # For existing tweets, assuming tweet text the same, so don't re-process
        # Update only the retweet and favorite counts
        print("Old Tweet")
        updatedtweet = items[0]
        updatedtweet["retweet_count"] = tweet["public_metrics"]["retweet_count"]
        updatedtweet["favorite_count"] = tweet["public_metrics"]["like_count"]
        tweet = updatedtweet
        user_obj = None  # don't need to re-insert the user if already seen before
        new_tweet = False
    elif not tweet["text"].lower().startswith("rt "):
        print("New Tweet")
        new_tweet = True
        tweet["originalid"] = tweet["id"]
        tweet["id"] = str(int(tweet["id"]) + abs(hash(topic)))  # artificially creating our own ID
        tweet["topickey"] = topic
        tweet["subtopic"] = subtopic
        tweet["month_year"] = str(str(parse(tweet["created_at"]).month) + "_" + str(parse(tweet["created_at"]).year))
        tweet["replies"] = []  # check if we want to keep this like this
        tmp_text = tweet["text"].replace('\n', '. ').replace('\r', '.').replace('..', '. ').replace(',.', '. ').replace(';.',
                                                                                                                   '. ').replace(
            '?.', '. ').replace('!.', '. ').replace(':.', '. ').lstrip('.').lstrip(' ')
        tmp_text = remove_emojis(tmp_text)
        tweet["processeed_text"] = tmp_text
        tweet["document_type"] = "tweet"
        if query_search:
            tweet["search_type"] = 'Topic Search'
            tweet["query"] = query
        else:
            tweet["search_type"] = 'User Search'
            tweet["searched_username"] = username
        tweet_text = tweet["processeed_text"]
        # get translation
        if not query_language:
            # will depend on language detection from the translator call
            translations, query_language = get_translation(tweet_text, target_languages)
        else:
            translations, _ = get_translation(tweet_text, target_languages)
        tweet["translations"] = translations
        # get named entities. Arabic only supports Person, Location and Organization entities, and seems to be poor, so doing NER on English text
        named_entity_obj = {}

#         if query_language != "en":
#             named_entities = get_ner(translations["en"])
#             org_language_entities = []
#             for ent in named_entities:
#                 org_language_ent = copy.deepcopy(ent)
#                 print(org_language_ent)
#                 org_language_ent["text"] = get_translation(ent["text"], query_language)[0][
#                     query_language]  # replace with original language text
#                 print(org_language_ent["text"])
#                 org_language_entities.append(org_language_ent)
#             named_entity_obj[query_language] = org_language_entities
        
        if query_language != "en":
            named_entities = get_ner(translations["en"])
            org_language_entities = []
            for ent in named_entities:
                org_language_ent = copy.deepcopy(ent)
                #print(org_language_ent)
                try:
                    org_language_ent["text"] = get_translation(ent["text"], query_language)[0][
                        query_language]  # replace with original language text
                    #print(org_language_ent["text"])
                except KeyError as e:
                    ##print("KeyError occurred while translating:", e)
                    org_language_ent["text"] = ent["text"]  # Assign original text if translation fails
                org_language_entities.append(org_language_ent)
            named_entity_obj[query_language] = org_language_entities
        else:
            named_entities = get_ner(tweet_text)  # list of objects where each object corresponds to an entity

            # add location information from azure maps
            named_entities_with_location = []
            for ent in named_entities:
                new_ent = copy.deepcopy(ent)
                if ent["category"] == "Location" and ent["subcategory"] == "GPE":
                    ent_text = ent["text"]
                    if ent_text not in regions:
                        # pass to Azure Maps to get country
                        r_json = get_maps_response(ent_text)
                        if r_json:  # i.e. got a response
                            if r_json["summary"]["numResults"] > 0:
                                if "address" in r_json['results'][0].keys():
                                    top_match = r_json['results'][0]["address"]
                                    if "country" in top_match.keys() and "countryCode" in top_match.keys():
                                        # there is a location detected, so get the country
                                        country = top_match["country"]
                                        country_code = top_match["countryCode"]
                                        new_ent["country_azuremaps"] = country
                                        new_ent["country_code_azuremaps"] = country_code
                named_entities_with_location.append(new_ent)
            named_entity_obj["en"] = named_entities_with_location

            

            for language in target_languages:
                if language != "en":
                    tmp_entities = []
                    for ent in named_entities:
                        tmp_ = copy.deepcopy(ent)
                        tmp_["text"] = get_translation(ent["text"], language)[0][language]
                        tmp_entities.append(tmp_)
                    named_entity_obj[language] = tmp_entities

        tweet["named_entities"] = named_entity_obj
        # Extract routes using get_routes_from_text() function
        routes = get_routes_from_text(tweet_text)
        tweet["routes"] = routes

        # get sentiment. Not supported for Arabic, so do on English. No need to translate back
        if query_language != "en":
            sentiment, sentiment_score = get_sentiment(translations["en"])
        else:
            sentiment, sentiment_score = get_sentiment(tweet_text)
        tweet["sentiment"] = {"sentiment": sentiment, "score": sentiment_score}
        user_obj = tweet["User"]
        # print(user_obj)
        user_obj["topickey"] = topic
        user_obj["id"] = user_obj["id"]
        user_obj["document_type"] = "user"
        user_obj['inserted_to_CosmosDB_at'] = at
        user_obj['inserted_to_CosmosDB_ts'] = ts
        user_obj["month_year"] = str(str(parse(user_obj["created_at"]).month) + "_" + str(parse(user_obj["created_at"]).year))

        try:
            user_location = tweet["User"]["location"]
        except KeyError:
            user_location = ""

        # user_location = tweet["User"]["location"]
        if user_location != "" and user_location not in regions:
            r_json = get_maps_response(user_location)
            if r_json:  # i.e. got a response
                if r_json["summary"]["numResults"] > 0:
                    # there is a location detected, so get the country
                    if "address" in r_json['results'][0].keys():
                        top_match = r_json['results'][0]["address"]
                        if "country" in top_match.keys() and "countryCode" in top_match.keys():
                            country = top_match["country"]
                            country_code = top_match["countryCode"]
                            user_obj["country_azuremaps"] = country
                            user_obj["country_code_azuremaps"] = country_code
        tweet["userid"] = user_obj["id"]
    else:
        return None, None, False
    return tweet, user_obj, new_tweet

def process_tweets(topic="", query="", language="en", maxdays=365, maxtweets_persearch=1, user="", query_search=True, container=None, target_languages=[], username=""):
    print("Working on topic: " + topic)
    end_date = datetime.utcnow() - timedelta(days=maxdays)
    all_tweets = []
    all_users = []
    count = 0
    # Reference: https://docs.tweepy.org/en/stable/api.html#API.search
    if query_search:
    # searching based on the query string
        if language:
            results = tweepy_client.search_recent_tweets(
                    query=query,
                    tweet_fields=[
                        'attachments',
                        'author_id',
                        'context_annotations',
                        'conversation_id',
                        'created_at',
                        'entities',
                        'geo',
                        
                        'id',
                        'in_reply_to_user_id',
                        'lang',
                        'note_tweet',
                        'possibly_sensitive',
                        'public_metrics',
                        'referenced_tweets',
                        'reply_settings',
                        'source',
                        'text'
                    ],
                    expansions=[
                        'author_id',
                        'referenced_tweets.id',
                        'referenced_tweets.id.author_id',
                        'entities.mentions.username',
                        'entities.note.mentions.username',
                        'attachments.media_keys',
                        'in_reply_to_user_id',
                        'geo.place_id',            ],
                    media_fields = ["url","height","width","public_metrics","media_key","preview_image_url"], 
                    user_fields = ["id", "name", "public_metrics", "username", "verified","url","profile_image_url","created_at","description","entities","location","pinned_tweet_id","verified_type"] ,
                    place_fields = ["contained_within","country","country_code","geo","name","place_type","full_name"], 

                max_results = num_tweets,
                since_id = since_id
                )

            # Get users list from the includes object
            try:
                users = {u["id"]:u.data for u in results.includes['users']}
            except KeyError:
                users = {}
            try:
                allMedia = {m["media_key"]: m.data for m in results.includes['media']}
            except KeyError:
                allMedia = {}    
            try:
                places = {p["id"]: p.data for p in results.includes['places']}
            except KeyError:
                places = {}


            if results.data is not None:
                for status in results.data:
                    count += 1
                    author_id = status['author_id']

                    media_key = None
                    if 'entities' in status and 'urls' in status['entities'] and len(status['entities']['urls']) > 1:
                        if 'media_key' in status['entities']['urls'][1]:
                            media_key = status['entities']['urls'][1]['media_key']

                    place_id = None
                    if 'geo' in status and 'place_id' in status['geo']:
                        place_id = status['geo']['place_id']

                    user = users.get(author_id)
                    media = allMedia.get(media_key)
                    place = places.get(place_id)

                    status = json.dumps(status.data)
                    status = json.loads(status)
                    status['User'] = user
                    status['Media'] = media
                    status['Place'] = place
                    status['tweet_url'] = "https://twitter.com/" + str(status['User']['username']) + "/status/" + str(status['id'])
                    status['query'] = query
                    
                    tweet_obj, user_obj, new_tweet = build_entry(topic, status, query_search, container, language, target_languages)
                    print(new_tweet)
                    all_tweets.append(tweet_obj)
                    if new_tweet:
                        all_users.append(user_obj)
                    if datetime.strptime(status['created_at'], "%Y-%m-%dT%H:%M:%S.%fZ") < end_date:
                        break
            else:
                print("No results found.")
            print("Found "+str(count) +" tweets for query: "+ query)
            return all_tweets, all_users
        else: # search for tweets regardless of tweet language
            print("Start search")
            results = tweepy_client.search_recent_tweets(
                    query=query,
                    tweet_fields=[
                        'attachments',
                        'author_id',
                        'context_annotations',
                        'conversation_id',
                        'created_at',
                        'entities',
                        'geo',
                        
                        'id',
                        'in_reply_to_user_id',
                        'lang',
                        'note_tweet',
                        'possibly_sensitive',
                        'public_metrics',
                        'referenced_tweets',
                        'reply_settings',
                        'source',
                        'text'
                    ],
                    expansions=[
                        'author_id',
                        'referenced_tweets.id',
                        'referenced_tweets.id.author_id',
                        'entities.mentions.username',
                        'entities.note.mentions.username',
                        'attachments.media_keys',
                        'in_reply_to_user_id',
                        'geo.place_id',            ],
                    media_fields = ["url","height","width","public_metrics","media_key","preview_image_url"], 
                    user_fields = ["id", "name", "public_metrics", "username", "verified","url","profile_image_url","created_at","description","entities","location","pinned_tweet_id","verified_type"],
                    place_fields = ["contained_within","country","country_code","geo","name","place_type","full_name"], 

                max_results = num_tweets,
                since_id = since_id
                )

            print("After search/results var")

            # Get users list from the includes object
            try:
                users = {u["id"]:u.data for u in results.includes['users']}
            except KeyError:
                users = {}
            try:
                allMedia = {m["media_key"]: m.data for m in results.includes['media']}
            except KeyError:
                allMedia = {}    
            try:
                places = {p["id"]: p.data for p in results.includes['places']}
            except KeyError:
                places = {}


            if results.data is not None:
                for status in results.data:
                    count += 1
                    author_id = status['author_id']

                    media_key = None
                    if 'entities' in status and 'urls' in status['entities'] and len(status['entities']['urls']) > 1:
                        if 'media_key' in status['entities']['urls'][1]:
                            media_key = status['entities']['urls'][1]['media_key']

                    place_id = None
                    if 'geo' in status and 'place_id' in status['geo']:
                        place_id = status['geo']['place_id']

                    user = users.get(author_id)
                    media = allMedia.get(media_key)
                    place = places.get(place_id)

                    status = json.dumps(status.data)
                    status = json.loads(status)
                    status['User'] = user
                    status['Media'] = media
                    status['Place'] = place
                    status['tweet_url'] = "https://twitter.com/" + str(status['User']['username']) + "/status/" + str(status['id'])
                    status['query'] = query
                    tweet_obj, user_obj, new_tweet = build_entry(topic, status, query_search, container, query_language=None, target_languages=target_languages)
                    all_tweets.append(tweet_obj)
                    all_users.append(user_obj)
                    if datetime.strptime(status['created_at'], "%Y-%m-%dT%H:%M:%S.%fZ") < end_date:
                        break 
            else:
                print("No results found.")
            print("Found "+str(count) +" tweets for query: "+ query)
            return all_tweets, all_users
    else:
    # getting tweets by user
        print("getting tweets by user")
        results = tweepy_client.get_users_tweets(
                    id=user,
                    tweet_fields=[
                        'attachments',
                        'author_id',
                        'context_annotations',
                        'conversation_id',
                        'created_at',
                        'entities',
                        'geo',
                        
                        'id',
                        'in_reply_to_user_id',
                        'lang',
                        'note_tweet',
                        'possibly_sensitive',
                        'public_metrics',
                        'referenced_tweets',
                        'reply_settings',
                        'source',
                        'text'
                    ],
                    expansions=[
                        'author_id',
                        'referenced_tweets.id',
                        'referenced_tweets.id.author_id',
                        'entities.mentions.username',
                        'entities.note.mentions.username',
                        'attachments.media_keys',
                        'in_reply_to_user_id',
                        'geo.place_id',            ],
                    media_fields = ["url","height","width","public_metrics","media_key","preview_image_url"], 
                    user_fields = ["id", "name", "public_metrics", "username", "verified","url","profile_image_url","created_at","description","entities","location","pinned_tweet_id","verified_type"],
                    place_fields = ["contained_within","country","country_code","geo","name","place_type","full_name"], 

                max_results = num_tweets,
                since_id = since_id
                )

        # Get users list from the includes object
        try:
            users = {u["id"]:u.data for u in results.includes['users']}
        except KeyError:
            users = {}
        try:
            allMedia = {m["media_key"]: m.data for m in results.includes['media']}
        except KeyError:
            allMedia = {}    
        try:
            places = {p["id"]: p.data for p in results.includes['places']}
        except KeyError:
            places = {}

        
        if results.data is not None:
            for status in results.data:
                #print(status)
                count += 1
                author_id = status['author_id']

                media_key = None
                if 'entities' in status and 'urls' in status['entities'] and len(status['entities']['urls']) > 1:
                    if 'media_key' in status['entities']['urls'][1]:
                        media_key = status['entities']['urls'][1]['media_key']

                place_id = None
                if 'geo' in status and 'place_id' in status['geo']:
                    place_id = status['geo']['place_id']

                User = users.get(author_id)
                media = allMedia.get(media_key)
                place = places.get(place_id)

                status = json.dumps(status.data)
                status = json.loads(status)
                status['User'] = User
                status['Media'] = media
                status['Place'] = place
                status['query'] = user
                tweet_obj, user_obj, new_tweet = build_entry(topic, status, query_search, container, query_language=None, target_languages=target_languages, username=username)
                all_tweets.append(tweet_obj)
                print(new_tweet)
                if new_tweet:
                    all_users.append(user_obj) # todo: optimize so not re-inserting the same user
                if datetime.strptime(status['created_at'], "%Y-%m-%dT%H:%M:%S.%fZ") < end_date:
                    break
        else:
            print("No results found.")            
        print("Found "+str(count) +" tweets for user: "+ user)
        return all_tweets, all_users


In [20]:
# queries = [
#     ('مصرف قطر المركزي -RT',"QCB"),
#     ('QCB Qatar -RT',"QCB"),
#      ('"Qatar Central Bank" -RT',"QCB"),
#     ('QCBQATRA',"QCB"),
#      ('محافظ مصرف قطر المركزي -RT',"المحافظ"),
#      ('بندر بن محمد بن سعود ال ثاني -RT',"المحافظ"),
#      ('Bandar bin Mohammed bin Saoud Al-Thani -RT',"المحافظ")]

In [21]:
#queries = [
#    ('("مطار حمد الدولي") -RT',"HIA")
#     ('("Hamad International Airport") -RT',"HIA"),
#     ('("HIAQatar") -RT',"HIA"),
#     ('(Qatar("Flights delay")) -RT',"Flights delay"),
#     ('(Qatar Airport ("customer service")) -RT',"customer service"),
#     ('(Qatar Airport (security)) -RT',"security"),
#     ('(Qatar Airport (cancellations)) -RT',"cancellations"),
#     ('(Qatar HIA ("customer service")) -RT',"customer service"),
#     ('(Qatar HIA (security)) -RT',"security"),
#     ('(Qatar HIA (cancellations)) -RT',"cancellations"),
#     ('(Qatar HIA (refunds)) -RT',"refunds"),
#     ('("Qatar Airways") -RT',"Qatar Airways"),
#     ("qatarairways","Qatar Airways")
# ]

In [27]:
queries = [
    ('مزاولة الانشطة بدون ترخيص السعودية -RT',"الانشطة"),
    
     ('مزاولة الانشطة بعد انتهاء صلاحية الترخيص السعودية -RT',"الانشطة"),
    ('مزاولة الانشطة الترخيص السعودية -RT',"الانشطة"),
    ('مزاولة انشطة ترخيص السعودية -RT',"الانشطة"),
    ('التعصب الرياضي السعودية -RT',"التعصب"),
    ('مخالفة ضوابط التواصل الاجتماعي السعودية -RT',"التواصل الاجتماعي"),
    ('ضوابط الانشطة الاعلامية السعودية -RT',"الانشطة الاعلامية"),
    ('محتوى اعلامي منسوخ السعودية -RT',"حقوق الملكية"),
    ('مخالفة حقوق ملكية فكرية السعودية -RT',"حقوق الملكية"),
    ('IPTV السعودية -RT',"حقوق الملكية"),
    ('تصوير ترخيص السعودية -RT',"تصوير بدون ترخيص"),
    ('تصنيف عمري افلام السعودية -RT',"لائحة التصنيف العمري"),
    ('تصنيف عمري العاب السعودية -RT',"لائحة التصنيف العمري"),
    ('تصنيف عمري برامج مرئية السعودية -RT',"لائحة التصنيف العمري"),
    ('اوقات عرض معتمدة سينما السعودية -RT',"اوقات العرض"),
    ('اوقات عرض معتمدة سينمائية السعودية -RT',"اوقات العرض"),
    ('تدني مستوى الخدمة السعودية -RT',"تدني مستوى الخدمة"),
    ('تدني مستوى الخدمة مرخصة السعودية -RT',"تدني مستوى الخدمة"),
    ('مستوى خدمة متدني السعودية -RT',"تدني مستوى الخدمة"),
    ('اساءة ثوابت الاسلام السعودية -RT',"ثوابت الشريعة"),
    ('اساءة ثوابت الشريعة السعودية -RT',"ثوابت الشريعة"),
    ('تجريح ثوابت الشريعة السعودية -RT',"ثوابت الشريعة"),
    ('تجريح ثوابت الاسلام السعودية -RT',"ثوابت الشريعة"),
    ('الاخلال النظام العام السعودية -RT',"مقتضيات المصلحة العامة"),
    ('الاخلال الامن الوطني السعودية -RT',"مقتضيات المصلحة العامة"),
    ('الاخلال المصلحة العامة السعودية -RT',"مقتضيات المصلحة العامة"),
    ('الضرر النظام العام السعودية -RT',"مقتضيات المصلحة العامة"),
    ('الضرر الامن الوطني السعودية -RT',"مقتضيات المصلحة العامة"),
    ('الضرر المصلحة العامة السعودية -RT',"مقتضيات المصلحة العامة"),
    ('علاقات الدول الخارجية السعودية -RT',"علاقات خارجية"),
    ('علاقات المملكة الخارجية السعودية -RT',"علاقات خارجية"),
    ('اثارة النعرات السعودية -RT',"الفرقة والكراهية"),
    ('اثارة الفرقة السعودية -RT',"الفرقة والكراهية"),
    ('اثارة الكراهية السعودية -RT',"الفرقة والكراهية"),
    ('محتوى اعلامي اداب عامة السعودية -RT',"الاداب العامة"),
    ('محتوى اداب عامة السعودية -RT',"الاداب العامة"),
    ('محتوى فواحش  السعودية -RT',"الاداب العامة"),
    ('محتوى رزائل  السعودية -RT',"الاداب العامة"),
    ('محتوى اداب لغة مبتذلة السعودية -RT',"الاداب العامة"),
    ('محتوى سحر السعودية -RT'," ترهيب وسحر وشعوذة"),
    ('محتوى شعوذة السعودية -RT'," ترهيب وسحر وشعوذة"),
    ('محتوى ترهيب السعودية -RT'," ترهيب وسحر وشعوذة"),
    ('محتوى الترهيب السعودية -RT'," ترهيب وسحر وشعوذة"),
    ('محتوى السحر و الشعوذة السعودية -RT'," ترهيب وسحر وشعوذة"),
    ('المساس حقوق المرأة السعودية -RT'," حقوق المرأة والطفل"),
    ('المساس حقوق الطفل السعودية -RT'," حقوق المرأة والطفل")



    
]

In [28]:
for query, topic in queries:
    # Set the since_datetime to be 7 days in the past from the current date and time
    since_datetime = datetime.utcnow() - timedelta(days=7)

    # Get the top 1 tweet from Cosmos DB that meets the criteria
    cosmos_query = f"SELECT TOP 1 c.originalid, c.created_at FROM c WHERE c.document_type = 'tweet' and c.query= '{query}' and c.created_at > '{since_datetime.isoformat()}' ORDER BY c.created_at DESC"
    items = list(tweet_container_client.query_items(cosmos_query, enable_cross_partition_query=True)) 

    # Retrieve the tweet ID if available
    if items:
        since_id = str(items[0]['originalid'])
    else:
        # Handle the case when no valid tweet ID is available
        since_id = None

    if users == "": # query search
        all_tweets, all_users = process_tweets(topic, query, query_language, maxdays=max_days, maxtweets_persearch=num_tweets, user="", query_search=True, container=tweet_container_client, target_languages=target_languages)
        update_cosmos(all_tweets, tweet_container_client) # insert tweets
        #update_cosmos(all_users, tweet_container_client) # insert users
    else: # user search
        for usr in userslist:
            all_tweets, all_users = process_tweets(topic, user=usr, maxdays=max_days, maxtweets_persearch=num_tweets, query_search=False, container=tweet_container_client, target_languages=target_languages, username=usr)
            update_cosmos(all_tweets, tweet_container_client) # insert tweets
            #update_cosmos(all_users, tweet_container_client) # insert users


Working on topic: الانشطة
Start search
After search/results var
No results found.
Found 0 tweets for query: مزاولة الانشطة بدون ترخيص السعودية -RT
Inserted data to cosmos
Working on topic: الانشطة
Start search
After search/results var
No results found.
Found 0 tweets for query: مزاولة الانشطة بعد انتهاء صلاحية الترخيص السعودية -RT
Inserted data to cosmos
Working on topic: الانشطة
Start search
After search/results var
No results found.
Found 0 tweets for query: مزاولة الانشطة الترخيص السعودية -RT
Inserted data to cosmos
Working on topic: الانشطة
Start search
After search/results var
No results found.
Found 0 tweets for query: مزاولة انشطة ترخيص السعودية -RT
Inserted data to cosmos
Working on topic: التعصب
Start search
After search/results var
No results found.
Found 0 tweets for query: التعصب الرياضي السعودية -RT
Inserted data to cosmos
Working on topic: التواصل الاجتماعي
Start search
After search/results var
No results found.
Found 0 tweets for query: مخالفة ضوابط التواصل الاجتماعي الس

In [29]:
all_tweets

[]